In [59]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np 

In [60]:
#url='https://www.mscorecard.com/mscorecard/showcourse.php?cid=141521162939611337'
url='https://www.mscorecard.com/mscorecard/showcourse.php?cid=14152075278110428'

In [61]:
#dfs = pd.read_html('https://www.mscorecard.com/mscorecard/showcourse.php?cid=141521162939611337')
df = pd.read_html(url)[0]

In [62]:
#dfs

In [63]:
#Create a handle, page, to handle the contents of the website
page = requests.get(url)

In [64]:
#Store the contents of the website under doc
doc = lh.fromstring(page.content)

In [65]:
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
#Test for the number of holes
if (len(tr_elements)) == 22:
    h=['2','3','4','5','6','7','8','9','10']
    ms=12
    mcr=13
    ws=14
    wcr=15
if (len(tr_elements)) == 33:
    h=['2','3','4','5','6','7','8','9','10','12','13','14','15','16','17','18','19','20']
    ms=23
    mcr=24
    ws=25
    wcr=26

In [66]:
#print (tr_elements)

In [67]:
#Check the length of the first 12 rows
#[len(T) for T in tr_elements[:22]]

In [68]:
#Create empty list
tees=[]
yards=[]
holes=[]
col=[]
par=[]
w_par=[]
m_hcp=[]
w_hcp=[]
m_slope=[]
m_rating=[]
w_slope=[]
w_rating=[]
i=0
j=2

In [69]:
#For each row, store each first element (header) and an empty list
for t in tr_elements[1]:
    i+=1
    name=t.text_content()
    #print ('%d:"%s"'%(i,name))
    #print(name)
    if name not in ['Hole','Par','Hcp']:
        tees.append(name)
    col.append((name,[]))
print('Tees', tees)

Tees ['White', 'Red']


In [70]:
#  front 9 and back 9 yardages
def get_value(index):
    if (len(tr_elements)) == 22:
        return list(np.concatenate([df.iloc[:,index][2:11].to_numpy()]))
    if (len(tr_elements)) == 33:
        return list(np.concatenate([df.iloc[:,index][2:11].to_numpy(), df.iloc[:,index][12:21].to_numpy()]))
yards = []
for i in range(3,len(tees)+3):
    yards.append(get_value(i))
print('Yards', yards) 

Yards [['119', '480', '243', '100', '324', '248', '95', '283', '109'], ['114', '465', '232', '97', '315', '238', '89', '272', '100']]


In [71]:
# front 9 and back 9 MEN pars and handicaps
for x in h:
    for t in (tr_elements[int(x)]):
        name=t.text_content()
        holes.append(name)
    par.append(holes[1])
    m_hcp.append(holes[2])
    holes.clear()
print('Men par', par)
print('Men Handicap', m_hcp)

Men par ['3', '5', '4', '3', '4', '4', '3', '4', '3']
Men Handicap ['-', '-', '-', '-', '-', '-', '-', '-', '-']


In [74]:
# front 9 and back 9 WOMEN pars and handicaps
for x in h:
    for t in (tr_elements[int(x)]):
        name=t.text_content()
        holes.append(name)
    w_par.append(holes[len(tees)+3])
    w_hcp.append(holes[len(tees)+4])
    holes.clear()
print('Women par', w_par)
print('Women Handicap', w_hcp)

Women par ['3', '5', '4', '3', '4', '4', '3', '4', '3', '3', '5', '4', '3', '4', '4', '3', '4', '3']
Women Handicap ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']


In [54]:
# men slope
for t in (tr_elements[ms]):
    name=t.text_content()
    holes.append(name)
for i in range(len(tees)):
    i+=1
    m_slope.append(holes[i])
holes.clear()
print('Men slope', m_slope)

Men slope ['88', '99']


In [55]:
# men rating
for t in (tr_elements[mcr]):
    name=t.text_content()
    holes.append(name)
for i in range(len(tees)):
    i+=1
    m_rating.append(holes[i])
holes.clear()
print('Men rating', m_rating)

Men rating ['30.6', '31.7']


In [56]:
# women slope
for t in (tr_elements[ws]):
    name=t.text_content()
    holes.append(name)
for i in range(len(tees)):
    i+=1
    w_slope.append(holes[i])
holes.clear()
print('Women slope', w_slope)

Women slope ['-', '-']


In [57]:
# women rating
for t in (tr_elements[wcr]):
    name=t.text_content()
    holes.append(name)
for i in range(len(tees)):
    i+=1
    w_rating.append(holes[i])
holes.clear()
print('Women rating',  w_rating)

Women rating ['-', '-']
